<a href="https://colab.research.google.com/github/kirbu123/FilmGeneration/blob/develop/FilmGenTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import time
from datetime import datetime
from tqdm import tqdm
import pandas as pd
from google.colab import drive
import os
import math
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
import numpy as np
from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedKFold, KFold
from sklearn.multiclass import OneVsRestClassifier
from PIL import Image
import requests
from io import BytesIO
from bs4 import BeautifulSoup
import pandas as pd
import csv
from urllib.request import urlopen
import multiprocessing

In [ ]:
!git clone https://github.com/huggingface/diffusers
!cd diffusers
!pip install -e .

Cloning into 'diffusers'...
remote: Enumerating objects: 38856, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 38856 (delta 119), reused 152 (delta 88), pack-reused 38633
Receiving objects: 100% (38856/38856), 25.78 MiB | 11.82 MiB/s, done.
Resolving deltas: 100% (28701/28701), done.
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.5 MB/s eta 0:00:00


In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [ ]:
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir, repo_type="dataset",
    ignore_patterns=".gitattributes",
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

'/content/dog'

In [ ]:
!cd /content
!mkdir -p model
!pwd

/content


In [ ]:
!pip install diffusers

In [6]:
!pip uninstall diffusers

Found existing installation: diffusers 0.21.3
Uninstalling diffusers-0.21.3:
  Would remove:
    /usr/local/bin/diffusers-cli
    /usr/local/lib/python3.10/dist-packages/diffusers-0.21.3.dist-info/*
    /usr/local/lib/python3.10/dist-packages/diffusers/*
Proceed (Y/n)? y
  Successfully uninstalled diffusers-0.21.3


In [9]:
!pip show diffusers

In [10]:
!export MODEL_NAME="CompVis/stable-diffusion-v1-4"
!export INSTANCE_DIR="/content/dog"
!export OUTPUT_DIR="/content/model"

!accelerate launch /content/diffusers/examples/dreambooth/train_dreambooth.py \
  --pretrained_model_name_or_path="CompVis/stable-diffusion-v1-4"  \
  --instance_data_dir="/content/dog" \
  --output_dir="/content/model" \
  --instance_prompt="a photo of sks dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=400

Traceback (most recent call last):
  File "/content/diffusers/examples/dreambooth/train_dreambooth.py", line 46, in <module>
    import diffusers
ModuleNotFoundError: No module named 'diffusers'
Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/accelerate_cli.py", line 47, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py", line 986, in launch_command
    simple_launcher(args)
  File "/usr/local/lib/python3.10/dist-packages/accelerate/commands/launch.py", line 628, in simple_launcher
    raise subprocess.CalledProcessError(returncode=process.returncode, cmd=cmd)
subprocess.CalledProcessError: Command '['/usr/bin/python3', '/content/diffusers/examples/dreambooth/train_dreambooth.py', '--pretrained_model_name_or_path=CompVis/stable-diffusion-v1-4', '--instance_data_dir=/content/dog', '--output_dir=/conten

In [ ]:
from diffusers import *
import torch

model_id = "path-to-your-trained-model"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

prompt = "A photo of sks dog in a bucket"
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

image.save("dog-bucket.png")